# Bike Rental Prediction

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Function to evaluate model performance
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    # Handle zero values in y_true to avoid division by zero in MAPE calculation
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    if len(y_true_filtered) > 0:
        mape = mean_absolute_percentage_error(y_true_filtered, y_pred_filtered) * 100
    else:
        mape = np.nan
    
    print(f"{model_name} Performance:")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R² Score: {r2:.4f}")
    print(f"MAPE: {mape:.2f}%\n")
    
    return {
        'model': model_name,
        'mse': mse,
        'rmse': rmse,
        'mae': mae, 
        'r2': r2,
        'mape': mape
    }

In [3]:
# Load the datasets
train_data = pd.read_csv('regression-dataset-train.csv')
test_data = pd.read_csv('regression-dataset-test-unlabeled.csv')

In [4]:
# Display basic information about the training dataset
print("Training dataset info:")
print(f"Shape: {train_data.shape}")
print("\nFirst few rows:")
print(train_data.head())
print("\nData types:")
print(train_data.dtypes)
print("\nSummary statistics:")
print(train_data.describe())
print("\nMissing values:")
print(train_data.isnull().sum())

Training dataset info:
Shape: (510, 14)

First few rows:
    id        date  season_id  year  month  is_holiday  weekday  \
0  577  31-07-2019          3     1      7           0        2   
1  427  03-03-2019          1     1      3           0        6   
2  729  30-12-2019          1     1     12           0        0   
3  483  28-04-2019          2     1      4           0        6   
4  112  22-04-2018          2     0      4           0        5   

   is_workingday  weather_condition  temperature  feels_like_temp  humidity  \
0              1                  1    29.246653          33.1448   70.4167   
1              0                  2    16.980847          20.6746   62.1250   
2              0                  1    10.489153          11.5850   48.3333   
3              0                  2    15.443347          18.8752   48.9583   
4              1                  2    13.803347          16.0977   72.9583   

   wind_speed  total_users  
0   11.083475         7216  
1   10.

## Data Exploration and Feature Engineering

In [5]:
# Convert date to datetime format
train_data['date'] = pd.to_datetime(train_data['date'], format='%d-%m-%Y')
test_data['date'] = pd.to_datetime(test_data['date'], format='%d-%m-%Y')

# Enhanced Feature Engineering - focusing on the most important features from previous run
for df in [train_data, test_data]:
    # Time-based features
    df['day'] = df['date'].dt.day
    df['day_of_year'] = df['date'].dt.dayofyear
    df['quarter'] = df['date'].dt.quarter
    
    # The previous analysis showed temperature is by far the most important feature
    # Create better temperature-related features
    df['temp_squared'] = df['temperature'] ** 2
    df['temp_cubed'] = df['temperature'] ** 3
    
    # Weather and temperature interactions
    df['weather_temp'] = df['weather_condition'] * df['temperature']
    df['temp_humidity_interaction'] = df['temperature'] * df['humidity']
    df['temp_humidity_scaled'] = df['temperature'] * (df['humidity'] / 100)
    df['feels_temp_humidity'] = df['feels_like_temp'] * (df['humidity'] / 100)
    
    # Temperature difference (a significant feature from previous run)
    df['temp_diff'] = df['temperature'] - df['feels_like_temp']
    df['temp_diff_sq'] = df['temp_diff'] ** 2
    
    # Month and season interactions (important in previous run)
    df['month_season'] = df['month'] * df['season_id']
    df['season_temp'] = df['season_id'] * df['temperature']
    
    # Improved cyclic encodings for seasonal patterns
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    df['day_sin'] = np.sin(2 * np.pi * df['day']/31)
    df['day_cos'] = np.cos(2 * np.pi * df['day']/31)
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday']/7)
    df['weekday_cos'] = np.cos(2 * np.pi * df['weekday']/7)
    
    # Day type features - Year was very important in previous run
    df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)
    df['peak_season'] = ((df['month'] >= 6) & (df['month'] <= 9)).astype(int)
    df['season_weekend'] = df['is_weekend'] * df['season_id']
    df['year_season'] = df['year'] * df['season_id']
    
    # High wind indicator - create thresholds based on data
    df['high_wind'] = (df['wind_speed'] > 15).astype(int)
    df['extreme_weather'] = (df['weather_condition'] >= 2).astype(int)
    
    # Log transform for potential non-linear relationships
    df['log_temp'] = np.log1p(df['temperature'] - df['temperature'].min() + 1)

# Feature selection - Focus on temperature, year, season as they were most important
# Extract key features for analysis
key_features = ['temperature', 'year', 'season_id', 'weather_temp', 'feels_like_temp', 
                'humidity', 'day_of_year', 'month_season', 'wind_speed', 
                'temp_humidity_interaction', 'temp_diff', 'month_sin', 'month_cos']

# Visualize correlation of key features with target
plt.figure(figsize=(14, 10))
key_corr = pd.concat([train_data[key_features], train_data['total_users']], axis=1).corr()['total_users'].sort_values(ascending=False)
sns.barplot(x=key_corr.values[1:], y=key_corr.index[1:])
plt.title('Correlation with Total Users (Key Features)')
plt.tight_layout()
plt.savefig('key_features_correlation.png')
plt.close()

# Visualize temperature vs total users with improved trend
plt.figure(figsize=(12, 8))
sns.scatterplot(x='temperature', y='total_users', hue='year', data=train_data, alpha=0.7)
sns.regplot(x='temperature', y='total_users', data=train_data, scatter=False, color='red')
plt.title('Temperature vs Total Users with Year Grouping')
plt.savefig('temp_users_by_year.png')
plt.close()

## Model Training and Evaluation


In [6]:
# Split the data
X = train_data.drop(['id', 'date', 'total_users'], axis=1)
y = train_data['total_users']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Select categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numerical_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_cols)
    ])

# Define models for ensemble
gb_model = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=3,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

xgb_model = xgb.XGBRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.9,
    random_state=42
)

lgb_model = lgb.LGBMRegressor(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=3,
    num_leaves=31,
    random_state=42,
    verbose=-1  # Suppress verbose output
)

# Create ensemble model
ensemble = VotingRegressor(
    estimators=[
        ('gb', gb_model),
        ('xgb', xgb_model),
        ('lgb', lgb_model)
    ],
    weights=[0.4, 0.35, 0.25]  # Weight based on previous performance, with GB having highest weight
)

# Create pipeline with ensemble
ensemble_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', ensemble)
])

# Create pipeline with gradient boosting (our previously best model)
gb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', gb_model)
])

# Function to perform cross-validation with multiple metrics
def cross_validate_with_metrics(pipeline, X, y, cv=5):
    print(f"Performing {cv}-fold cross-validation...")
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    
    mse_scores = []
    rmse_scores = []
    mae_scores = []
    r2_scores = []
    mape_scores = []
    
    for train_idx, val_idx in kf.split(X):
        X_train_cv, X_val_cv = X.iloc[train_idx], X.iloc[val_idx]
        y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]
        
        pipeline.fit(X_train_cv, y_train_cv)
        y_pred_cv = pipeline.predict(X_val_cv)
        
        mse_scores.append(mean_squared_error(y_val_cv, y_pred_cv))
        rmse_scores.append(np.sqrt(mean_squared_error(y_val_cv, y_pred_cv)))
        mae_scores.append(mean_absolute_error(y_val_cv, y_pred_cv))
        r2_scores.append(r2_score(y_val_cv, y_pred_cv))
        
        # Handle zero values in y_true to avoid division by zero in MAPE calculation
        mask = y_val_cv != 0
        y_val_cv_filtered = y_val_cv[mask]
        y_pred_cv_filtered = y_pred_cv[mask]
        if len(y_val_cv_filtered) > 0:
            mape_scores.append(mean_absolute_percentage_error(y_val_cv_filtered, y_pred_cv_filtered) * 100)
        else:
            mape_scores.append(np.nan)
    
    print(f"Cross-validation results:")
    print(f"Mean MSE: {np.mean(mse_scores):.2f} (±{np.std(mse_scores):.2f})")
    print(f"Mean RMSE: {np.mean(rmse_scores):.2f} (±{np.std(rmse_scores):.2f})")
    print(f"Mean MAE: {np.mean(mae_scores):.2f} (±{np.std(mae_scores):.2f})")
    print(f"Mean R²: {np.mean(r2_scores):.4f} (±{np.std(r2_scores):.4f})")
    print(f"Mean MAPE: {np.mean(mape_scores):.2f}% (±{np.std(mape_scores):.2f}%)")
    print()
    
    return {
        'mse': np.mean(mse_scores),
        'rmse': np.mean(rmse_scores),
        'mae': np.mean(mae_scores),
        'r2': np.mean(r2_scores),
        'mape': np.mean(mape_scores)
    }

# Evaluate both models with cross-validation
print("Evaluating Gradient Boosting model with cross-validation:")
gb_cv_results = cross_validate_with_metrics(gb_pipeline, X, y, cv=5)

print("Evaluating Ensemble model with cross-validation:")
ensemble_cv_results = cross_validate_with_metrics(ensemble_pipeline, X, y, cv=5)

# Train and evaluate on validation set
print("Training and evaluating on validation set...")
gb_pipeline.fit(X_train, y_train)
y_pred_gb = gb_pipeline.predict(X_val)
gb_results = evaluate_model(y_val, y_pred_gb, "Gradient Boosting")

ensemble_pipeline.fit(X_train, y_train)
y_pred_ensemble = ensemble_pipeline.predict(X_val)
ensemble_results = evaluate_model(y_val, y_pred_ensemble, "Ensemble")

# Choose the best model
best_model_pipeline = gb_pipeline if gb_results['mse'] < ensemble_results['mse'] else ensemble_pipeline
best_model_name = "Gradient Boosting" if gb_results['mse'] < ensemble_results['mse'] else "Ensemble"
print(f"\nSelected model: {best_model_name}")

Evaluating Gradient Boosting model with cross-validation:
Performing 5-fold cross-validation...
Cross-validation results:
Mean MSE: 455688.48 (±97663.00)
Mean RMSE: 671.05 (±73.37)
Mean MAE: 470.46 (±48.30)
Mean R²: 0.8758 (±0.0301)
Mean MAPE: 43.41% (±56.73%)

Evaluating Ensemble model with cross-validation:
Performing 5-fold cross-validation...
Cross-validation results:
Mean MSE: 440839.49 (±107001.09)
Mean RMSE: 658.82 (±82.46)
Mean MAE: 457.53 (±56.53)
Mean R²: 0.8801 (±0.0307)
Mean MAPE: 42.63% (±55.74%)

Training and evaluating on validation set...
Gradient Boosting Performance:
MSE: 458707.56
RMSE: 677.28
MAE: 431.19
R² Score: 0.8513
MAPE: 14.43%

Ensemble Performance:
MSE: 448829.35
RMSE: 669.95
MAE: 421.20
R² Score: 0.8545
MAPE: 14.32%


Selected model: Ensemble


## Hyper Parameter Fine-Tuning

In [7]:
if best_model_name == "Gradient Boosting":
    # Define more refined parameter grid for GB
    param_grid = {
        'model__n_estimators': [150, 200, 250],
        'model__learning_rate': [0.05, 0.1, 0.15],
        'model__max_depth': [3, 4],
        'model__min_samples_split': [2, 5],
        'model__min_samples_leaf': [1, 2],
        'model__subsample': [0.8, 0.9, 1.0]
    }
    
    # Create pipeline for tuning
    tuning_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', GradientBoostingRegressor(random_state=42))
    ])
else:
    # If ensemble is best, fine-tune the weights
    param_grid = {
        'model__weights': [[0.4, 0.35, 0.25], [0.45, 0.3, 0.25], [0.35, 0.4, 0.25]]
    }
    
    # Create tuning pipeline for ensemble
    tuning_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', VotingRegressor(
            estimators=[
                ('gb', gb_model),
                ('xgb', xgb_model),
                ('lgb', lgb_model)
            ]
        ))
    ])

print(f"Fine-tuning {best_model_name} model...")
grid_search = GridSearchCV(
    tuning_pipeline,
    param_grid=param_grid,
    cv=3,  # Reduced CV to speed up tuning
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_search.fit(X, y)  # Use full training data

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV score: {-grid_search.best_score_:.2f} (MSE)")

# Final model with best parameters
final_pipeline = grid_search.best_estimator_

Fine-tuning Ensemble model...


/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.w

Best parameters: {'model__weights': [0.35, 0.4, 0.25]}
Best CV score: 457334.90 (MSE)


## Training Final Model

In [8]:
final_pipeline.fit(X, y)

# Feature importance analysis for GradientBoosting (if applicable)
if hasattr(final_pipeline.named_steps['model'], 'feature_importances_'):
    print("\nFeature importances:")
    importances = final_pipeline.named_steps['model'].feature_importances_
    feature_names = numerical_cols + categorical_cols
    
    feature_importance = pd.DataFrame({
        'Feature': feature_names[:len(importances)] if len(feature_names) >= len(importances) else feature_names + ['Unknown'] * (len(importances) - len(feature_names)),
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    print("Top 15 features by importance:")
    print(feature_importance.head(15))
    
    # Plot feature importance
    plt.figure(figsize=(12, 10))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
    plt.title(f'Feature Importance ({best_model_name})')
    plt.tight_layout()
    plt.savefig('final_feature_importance.png')
    plt.close()
elif isinstance(final_pipeline.named_steps['model'], VotingRegressor):
    # For ensemble, get feature importance from the first estimator (GB)
    if hasattr(final_pipeline.named_steps['model'].estimators_[0], 'feature_importances_'):
        print("\nFeature importances from Gradient Boosting (part of ensemble):")
        importances = final_pipeline.named_steps['model'].estimators_[0].feature_importances_
        feature_names = numerical_cols + categorical_cols
        
        feature_importance = pd.DataFrame({
            'Feature': feature_names[:len(importances)] if len(feature_names) >= len(importances) else feature_names + ['Unknown'] * (len(importances) - len(feature_names)),
            'Importance': importances
        }).sort_values('Importance', ascending=False)
        
        print("Top 15 features by importance:")
        print(feature_importance.head(15))


Feature importances from Gradient Boosting (part of ensemble):
Top 15 features by importance:
                Feature  Importance
33          year_season    0.472356
14         temp_squared    0.064798
36             log_temp    0.054112
7           temperature    0.050813
16         weather_temp    0.043942
8       feels_like_temp    0.040307
15           temp_cubed    0.039163
1                  year    0.035747
9              humidity    0.035517
23          season_temp    0.032538
10           wind_speed    0.025755
21         temp_diff_sq    0.015578
6     weather_condition    0.014072
12          day_of_year    0.011839
19  feels_temp_humidity    0.010171


## Generating Predictions

In [9]:
test_predictions = final_pipeline.predict(test_data.drop(['id', 'date'], axis=1))

# Create submission file
submission = pd.DataFrame({
    'id': test_data['id'],
    'label': test_predictions
})

# Ensure predictions are non-negative
submission['label'] = submission['label'].clip(0)

# Save to CSV
submission.to_csv('regression_predictions.csv', index=False)
print(f"Predictions saved to 'regression_predictions.csv'")

# Print submission summary statistics
print("\nSubmission summary statistics:")
print(submission['label'].describe())

Predictions saved to 'regression_predictions.csv'

Submission summary statistics:
count     220.000000
mean     4441.380711
std      1729.363251
min       771.264054
25%      3321.255112
50%      4508.747255
75%      5797.807723
max      8096.700788
Name: label, dtype: float64
